In [1]:
#x02_chewc

In [2]:
import torch
import matplotlib.pyplot as plt
from fastcore.basics import patch
import uuid

In [16]:
import torch

class Genome:
    def __init__(self, n_chr, n_loci):
        self.ploidy = 2
        self.n_chr = n_chr
        self.n_loci = n_loci
        self.shape = (self.ploidy, self.n_chr, self.n_loci)
        
class Population:
    def __init__(self, genome, haplotypes=None, device='cpu'):
        self.g = genome
        self.device = device
        if haplotypes:
            self.haplotypes=haplotypes
        else:
            self.haplotypes = self._create_random_haplotypes(500)
            
        self._calculate_dosages()
            
    def _calculate_dosages(self):
        self.dosages = self.haplotypes.sum(dim=1)
        
    def _create_random_haplotypes(self,num_individuals):
        return torch.randint(0, 2, (num_individuals, *self.g.shape), device=self.device)
    
    def __getitem__(self,index):
        return self.haplotypes[index]
    
class Trait:
    def __init__(self, genome, founder_population, target_mean, target_variance, device='cpu'):
        self.target_mean = target_mean
        self.target_variance = target_variance
        self.device = device
        random_effects = torch.randn(genome.n_chr, genome.n_loci, device=self.device)
        founder_scores = torch.einsum('kl,hjkl->h', random_effects, founder_population.haplotypes.float())
        founder_mean, founder_var = founder_scores.mean(), founder_scores.var()
        scaling_factors = torch.sqrt(self.target_variance / founder_var)
        random_effects *= scaling_factors
        self.effects = random_effects
        self.intercept = founder_mean - target_mean

In [17]:
G = Genome(50,10)
P = Population(genome=G)
P[0].shape

torch.Size([2, 50, 10])

In [5]:
T = Trait(G,P, 0, 1)

In [7]:
class Individual:
    def __init__(self, genome, haplotype=None, mother=None, father=None, descendants=0, source='founder', chewc=None, device='cpu'):
        self.genome = genome
        self.haplotype = None
        self.source = source
        self.descendents = []
        self.fitness = 0
        self.mother, self.father = mother, father
        self.device = device

        if self.source == 'founder':
            self.haplotype = self._generate_random_haplotype()
        else:
            self.haplotype = haplotype
        self.make_id()

    def __repr__(self):
        return f'Individual with ID: {self.id[:3]}'

    def make_id(self):
        self.id = uuid.uuid4().hex
